In [4]:
import cv2
import pytesseract
import time
import re
import speech_recognition as sr
import pyttsx3
import threading
import torch

In [5]:
# Configure Tesseract OCR (Update path if needed)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Initialize text-to-speech engine
engine = pyttsx3.init()
recognizer = sr.Recognizer()
stop_flag = False  # Flag to control thread execution

def text_to_speech(text):
    """Convert text to speech using pyttsx3."""
    engine.say(text)
    engine.runAndWait()

def is_valid_text(text):
    """Ensures detected text is meaningful and not just single letters or artifacts."""
    text = text.strip()
    if len(text) < 2 or re.fullmatch(r"[^A-Za-z]+", text):  # Reject short words & symbols
        return False
    return True

def listen_for_trigger():
    """Listen for the trigger word 'read' to activate OCR and TTS."""
    global stop_flag
    while not stop_flag:
        with sr.Microphone() as source:
            recognizer.adjust_for_ambient_noise(source)
            print("Listening for trigger word 'read'...")
            try:
                audio = recognizer.listen(source, timeout=20)
                spoken_text = recognizer.recognize_google(audio).lower()
                if "read" in spoken_text:
                    return True
            except sr.UnknownValueError:
                pass
            except sr.RequestError:
                print("Speech recognition service unavailable")
    return False

def process_frame(frame):
    """Process a video frame for OCR detection using GPU if available."""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Use GPU acceleration if available
    if torch.cuda.is_available():
        device = torch.device("cuda")
        tensor_frame = torch.tensor(gray, device=device)
        gray = tensor_frame.cpu().numpy()
    
    detected_text = pytesseract.image_to_string(gray)
    valid_words = [word for word in detected_text.split() if is_valid_text(word)]
    return " ".join(valid_words)

# Open webcam
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cap.set(cv2.CAP_PROP_FPS, 30)  # Optimize for smoother video
if not cap.isOpened():
    print("Error: Could not access the webcam.")
    exit()

print("Press 'q' to quit.")
trigger_thread = threading.Thread(target=listen_for_trigger)
trigger_thread.start()


Press 'q' to quit.


Listening for trigger word 'read'...
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Listening for trigger word 'read'...


In [6]:
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture video. Exiting...")
        break

    if listen_for_trigger():
        print("Trigger word detected. Capturing text...")
        full_text = process_frame(frame)

        if full_text:
            print(f"Detected Text: {full_text}")
            text_to_speech(full_text)
            time.sleep(2)  # Prevent repeating too fast

        # Draw red rectangle around detected text
        h, w, _ = frame.shape
        cv2.rectangle(frame, (50, 50), (w - 50, h - 50), (0, 0, 255), 3)
        cv2.putText(frame, full_text, (60, h - 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Display video feed
    cv2.imshow("Live OCR Feed - Say 'Read' to Capture", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        stop_flag = True
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
trigger_thread.join()

Listening for trigger word 'read'...
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Trigger word detected. Capturing text...
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Listening for trigger word 'read'...
Listening for trigger word 'read'...


: 